In [22]:
import os
import pandas as pd
# import numpy as np
from random import random as rnd

In [44]:
FILE = os.path.join(os.getcwd(), 'to_check_list.csv')
df = pd.read_csv(FILE, encoding='utf-8-sig')
df0 = df
df0

,库位,料品,品名,批号,可用数量,冻结数量,总数量
0,3A00,100101-35,太粮王泰国茉莉香米5*4,200101001,4,0,4
1,3A00,100502-35,（橙色）柬埔寨茉莉香米5*5,200617002,405,0,405
2,3A00,100604-35,曼哈浓香德莉进口香米5*5,200804009,3,0,3
3,3A00,100607-35,曼哈浓香精选小茉莉香米5*5,200805008,167,0,167
4,3A00,101301-31,太粮靓虾王香软米1*25,200813007,83,0,83
...,...,...,...,...,...,...,...
406,糙米线库位,107006-10,太粮良谷纪臻选东北大米10,200817013,1250,0,1250
407,糙米线库位,107090-25,香雪珍珠米（特通）25,200804019,598,0,598
408,糙米线库位,107090-25,香雪珍珠米（特通）25,200804033,40,0,40
409,糙米线库位,107090-25,香雪珍珠米（特通）25,200806014,67,0,67


In [57]:
tough = False
trial = 5
while not tough and trial > 0:
    df = df0

    display(f'#{trial}')
    trial -= 1
    df['delta'] = 0
   
    # wrong location or wrong batch
    p_wlb = 0.5
    r_wlb = 1.2 
    assert 0 <= p_wlb <= 1 and r_wlb >= 0
    for sku, df_sku in df.groupby(['料品']):
        if len(df_sku) > 2:
            for _ in range(len(df_sku)):    
                if rnd() < p_wlb:
                    _df = df_sku.sample(n=2)    # random pick 2 to switch
                    a, b = _df.index[0], _df.index[1]
                    d = int(df['总数量'][a] * rnd() * r_wlb)    # d > 0
                    d = min(df['总数量'][a] + df['delta'][a], df['总数量'][b] + df['delta'][b], d)    # avoid negative
                    df['delta'][a] -= d
                    df['delta'][b] += d
    display(len(df[df['delta'] != 0]))

    # black inventory
    p_bi = 0.05
    r_bi = 0.3
    assert 0 <= p_bi <= 1 and 0 < r_bi < 1
    locations = df.groupby('库位').groups.keys()    
    for sku_batch, df_sku_batch in df.groupby(['料品', '批号']):
        sku, batch = sku_batch
        locations_sku = df_sku_batch.groupby('库位').groups.keys()
        goals = list(set(locations) - set(locations_sku))
        for _ in range(len(df_sku_batch)):
            if rnd() < p_bi:
                _df = df_sku_batch.sample(n=1)    # random pick 1 to move out
                i = _df.index[0]
                d = int((_df['总数量'][i] + _df['delta'][i]) * rnd() * r_bi)
                if d > 0:
                    _df['库位'] = goals[int(len(goals) * rnd())]
                    _df[['可用数量', '冻结数量', '总数量']] = 0
                    _df['delta'] = d
                    df['delta'][i] -= d
                    df = pd.concat([df, _df], ignore_index=True)
    display(len(df[df['delta'] != 0]))


    # wrong amount
    p_wa = 0.1
    r_wa = 0.2
    assert 0 <= p_wa <= 1 and r_wa >= 0
    for i in df.index:
        if rnd() < p_wa:
            d = int(df['总数量'][i] * 2 * (rnd() - 0.5) * r_wa)
            if d < 0:
                df['delta'][i] = max(-df['总数量'][i], df['delta'][i] + d)    # avoid negative
    display(len(df[df['delta'] != 0]))


    # check whether it is tough enough
    df['check'] = df['总数量'] + df['delta']
    _insufficient = df[df['check'] < df['冻结数量']]
    _missed = df[df['check'] == 0]
    _frozen = df[(df['check'] > 0) & (df['check'] == df['冻结数量'])]
    _black = df[(df['可用数量'] + df['冻结数量'] + df['总数量']) == 0]
    if len(_insufficient) and len(_missed) and len(_frozen) and len(_black):
          tough = True

assert len(df[df['check'] < 0]) == 0
display(_insufficient, _missed, _frozen, _black)

'#5'

105

130

138

,库位,料品,品名,批号,可用数量,冻结数量,总数量,delta,check
311,4AP6,101601-10,（米酷）马坝银粘10,200817006,0,80,80,-46,34
343,4B02,101303-10,（靓虾）软米10,200814003,0,1498,1498,-633,865


,库位,料品,品名,批号,可用数量,冻结数量,总数量,delta,check
90,3AP1,100502-35,（橙色）柬埔寨茉莉香米5*5,200617002,39,0,39,-39,0
130,4A01,101420-25,米皇坊珍珠香米25,200806013,1,0,1,-1,0
154,4A04,101308-25,（信鲜）靓虾王香软米25,200815001,34,0,34,-34,0
171,4A05,103909-15,今西利哈旺油粘米15,200811004,23,0,23,-23,0
210,4AH0,100607-05,曼哈浓香精选小茉莉香米5,200621010,2,0,2,-2,0
337,4B02,100607-10,曼哈浓香精选小茉莉香米10,200814004,150,0,150,-150,0


,库位,料品,品名,批号,可用数量,冻结数量,总数量,delta,check
88,3A09,150101-35,鸣象（尊桂）油粘米5*5,200811011,124,6,130,-124,6
143,4A03,101301-10,太粮靓虾王香软米10,200816003,0,217,217,0,217
364,4B03,101307-35,（西关）靓虾王米5*5,200805003,0,1,1,0,1


,库位,料品,品名,批号,可用数量,冻结数量,总数量,delta,check
411,4AP6,100101-25,太粮王泰国茉莉香米25,200817005,0,0,0,12,12
412,4BH0,100607-10,曼哈浓香精选小茉莉香米10,200816004,0,0,0,191,191
413,4A07,101202-10,马坝龙三星油粘米10,200818007,0,0,0,20,20
414,4AP4,101302-10,经典靓虾王香软米10,200817019,0,0,0,127,127
415,3A02,101303-10,（靓虾）软米10,200816006,0,0,0,345,345
416,4AP2,101306-53,（红香）靓虾王米1.5,200526014,0,0,0,9,9
417,3AP1,101402-10,米皇坊小农粘米10,200818004,0,0,0,445,445
418,4B03,101420-15,米皇坊珍珠香米15,200817015,0,0,0,20,20
419,4A06,101601-15,（米酷）马坝银粘15,200812004,0,0,0,21,21
420,4AH0,104003-25,香满宾庭励福香米25,200725004,0,0,0,29,29


In [58]:
for i in df.index:
        if df['delta'][i] == 0:
            df['check'][i] = pd.NA
display(df.sample(n=20))
df.drop(columns=['delta']).to_csv(os.path.join(os.getcwd(), 'checked_list.csv'), index=False, encoding='utf-8-sig')

,库位,料品,品名,批号,可用数量,冻结数量,总数量,delta,check
161,4A04,104101-25,金稻鱼太粮家乐米25,200724028,90,0,90,-86,4
335,4B01,160102-25,帝皇鹰喜之粮米25,200815019,80,0,80,-2,78
251,4AH1,100601-31,(红色）新米红曼哈浓香泰国茉莉香米1*20,200604001,100,0,100,0,<NA>
1,3A00,100502-35,（橙色）柬埔寨茉莉香米5*5,200617002,405,0,405,0,<NA>
213,4AH0,100607-05,曼哈浓香精选小茉莉香米5,200621010-03,2,0,2,0,<NA>
92,3AP1,100601-10,(红色）新米红曼哈浓香泰国茉莉香米10,200729010,20,0,20,0,<NA>
73,3A09,101301-75,太粮靓虾王香软米0.5*50,200708002,10,7,17,0,<NA>
103,4A00,100101-35,太粮王泰国茉莉香米5*4,191119006,29,0,29,2,31
140,4A02,101601-15,（米酷）马坝银粘15,200812004,105,0,105,-21,84
391,4BH0,103911-10,靓当家太粮虾仔米10,200815002,55,0,55,0,<NA>
